In [2]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices(device_type="GPU")
tf.config.experimental.set_visible_devices(devices=gpus[0], device_type="GPU")
tf.config.experimental.set_memory_growth(device=gpus[0], enable=True)

In [4]:
import os 
os.getcwd()
#os.chdir('C:\\Users\\narayanan\\Desktop\\dataset')

'C:\\Users\\pruth\\Desktop\\New folder'

In [3]:
!pip install facenet_pytorch
!pip install mmcv

  Created wheel for mmcv: filename=mmcv-1.2.7-py2.py3-none-any.whl size=336567 sha256=dbff499b2a81b440f4b41d4ad6f18a73858cc808badfeba91f1ab1f0179152b3
  Stored in directory: c:\users\pruth\appdata\local\pip\cache\wheels\15\9c\42\c30611ff05230151d3c24da668d57afcedaffd95db49a87b8b
Successfully built mmcv
  Using cached facenet_pytorch-2.5.2-py3-none-any.whl (1.9 MB)
  Using cached torchvision-0.9.0-cp38-cp38-win_amd64.whl (852 kB)


ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\users\\pruth\\anaconda3\\Lib\\site-packages\\torch\\distributed\\nn\\api\\remote_module.py'
Consider using the `--user` option or check the permissions.



In [5]:
import os
import glob
import json
import torch
import cv2
from PIL import Image
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from facenet_pytorch import MTCNN

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Running on device: {device}')

Running on device: cpu


In [28]:
class FaceExtractor:
    def __init__(self, detector, n_frames=None, resize=None):
        """
        Parameters:
            n_frames {int} -- Total number of frames to load. These will be evenly spaced
                throughout the video. If not specified (i.e., None), all frames will be loaded.
                (default: {None})
            resize {float} -- Fraction by which to resize frames from original prior to face
                detection. A value less than 1 results in downsampling and a value greater than
                1 result in upsampling. (default: {None})
        """

        self.detector = detector
        self.n_frames = n_frames
        self.resize = resize
    
    def __call__(self, filename, save_dir):
        """Load frames from an MP4 video, detect faces and save the results.

        Parameters:
            filename {str} -- Path to video.
            save_dir {str} -- The directory where results are saved.
        """

        # Create video reader and find length
        v_cap = cv2.VideoCapture(filename)
        v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Pick 'n_frames' evenly spaced frames to sample
        if self.n_frames is None:
            sample = np.arange(0, v_len)
        else:
            sample = np.linspace(0, v_len - 1, self.n_frames).astype(int)

        # Loop through frames
        for j in range(v_len):
            success = v_cap.grab()
            if j in sample:
                # Load frame
                success, frame = v_cap.retrieve()
                if not success:
                    continue
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = Image.fromarray(frame)
                
                # Resize frame to desired size
                if self.resize is not None:
                    frame = frame.resize([int(d * self.resize) for d in frame.size])

                save_path = os.path.join(save_dir, f'{j}.png')

                self.detector([frame], save_path=save_path)

        v_cap.release()

In [29]:
face_detector = MTCNN(margin=14, keep_all=True, factor=0.5, device=device).eval()

In [30]:
face_extractor = FaceExtractor(detector=face_detector, n_frames=100)

In [36]:
#Define the directory path in which video is 
TRAIN_DIR = os.getcwd()+'\\test\\'
TRAIN_DIR

'C:\\Users\\pruth\\Desktop\\New folder\\test\\'

In [37]:
all_train_videos = glob.glob(os.path.join(TRAIN_DIR, '*.mp4'))
all_train_videos

['C:\\Users\\pruth\\Desktop\\New folder\\test\\001.mp4']

In [38]:
TMP_DIR =os.getcwd()+'\\temp'

In [39]:
with torch.no_grad():
    for path in tqdm(all_train_videos):
        file_name = path.split('/')[-1]

        save_dir = os.path.join(TMP_DIR, file_name.split(".")[0])

        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        # Detect all faces appear in the video and save them.
        face_extractor(path, save_dir)

In [44]:
import shutil
s = os.getcwd()+"\\Test\\001"
d = os.getcwd()+"\\Test\\Temp\\001"
shutil.move(s,d)

'C:\\Users\\pruth\\Desktop\\New folder\\Test\\Temp\\001'

In [45]:
path = os.getcwd()+'\\test\\Temp'
import cv2
test_data_X = list()
num1 = 256
num2 = 256
for fold in os.listdir(path):
    new_path = os.path.join(path, fold)
    c = 0
    for frames in os.listdir(new_path):
        if(c == 150):
            break
        c = c + 1
        path_a = os.path.join(new_path,frames)
        img = cv2.imread(path_a)
        new_img = cv2.resize(img, (num2, num1))
        test_data_X.append(new_img)

In [46]:
np.array(test_data_X).shape

(105, 256, 256, 3)

In [24]:
from keras.models import load_model

In [47]:
classifier4 = load_model('Mesonet_4_Model.h5')
test_data_X = np.array(test_data_X)
predictions = classifier4.predict(test_data_X)
result = []
for i in range(len(predictions)):
    if predictions[i] >=0.5:
        result.append(1)
    else:
        result.append(0)
if result.count(1)>=result.count(0):
    print("REAL")
else:
    print("FAKE")

FAKE
